In [4]:
import os
import sqlite3
import pandas as pd

# Run this to go up a level
os.chdir(os.path.dirname(os.path.abspath('')))

In [5]:
os.getcwd()

'/Users/nikkivanhandel/CS6400/mfgdb'

In [6]:
conn = sqlite3.connect("mfgdb")

In [7]:
builds_df = pd.read_sql_query("SELECT * FROM Builds", conn)
builds_df = builds_df.drop(0).reset_index(drop=True)
builds_df = builds_df.rename(columns={'Build ID:': 'BuildID'})

print("Builds Data:")
display(builds_df.head())

DatabaseError: Execution failed on sql 'SELECT * FROM Builds': no such table: Builds

In [3]:
build_df

NameError: name 'build_df' is not defined

In [ ]:
# Analysis Case: Find images which correspond to failed builds

# Use case: 

In [ ]:
# Analysis Case: Find images taken between date - date 

In [13]:

sensors_df = pd.read_sql_query("SELECT * FROM Sensors", conn)

print("Sensors:")
display(sensors_df.head())

Sensors:


,Serial,Name,DataKind,Model,Manufacturer,Resolution,Depth
0,1,cam1,PHX-200S-MC,optical image,Lucid Vision,5472x3648,12
1,2,cam2,PHX-032S-MC,optical image,Lucid Vision,2048x1536,12
2,3,laser power,daq-power,voltage,NIST,0.0001,32


In [14]:
sensor_data = pd.read_sql_query("SELECT * FROM SensorData", conn)
display(sensor_data.head())

,RecordID,BuildID,SensorSerial,Date,Time
0,1,B058,cam1,2023-01-06,15:36:40
1,2,B058,cam1,2023-01-06,15:36:47
2,3,B058,cam1,2023-01-06,15:36:54
3,4,B058,cam1,2023-01-06,15:41:50
4,5,B058,cam1,2023-01-06,15:41:59


In [ ]:
conn.close()